In [1]:
import pandas as pd
import numpy as np
import sys


In [2]:
try:
    import google.colab
    colab = True
    from google.colab import drive
    drive.mount('/content/drive')
    
    fine_tuned_filepath = '/content/drive/MyDrive/NLP_files/mybert.pickle'
    concepts_filepath = '/content/drive/MyDrive/NLP_files/concepts_strings_with_ids.csv'
    embeddings_output_filepath = '/content/drive/MyDrive/NLP_files/concepts_embeddings.csv'
    device_name = 'cuda'
    sys.path.append('/content/drive/MyDrive/NLP/bert_embeddings')
    print(sys.path[-1])
except ImportError:
    colab = False
    fine_tuned_filepath = '../models/mybert.pickle'
    concepts_filepath = '../data/concepts_strings_with_ids.csv'
    embeddings_output_filepath = '../results/embeddings/concepts_embeddings.csv'
    device_name = 'cpu'

In [3]:
#!pip install transformers

In [4]:
#!rm -R /content/drive/MyDrive/NLP
#!git clone https://github.com/hadaszm/NLP.git /content/drive/MyDrive/NLP

In [5]:
from transformers import BertTokenizer, BertForPreTraining, BertModel
import torch
from emb_helpers import * 

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_pretrained = BertModel.from_pretrained('bert-base-uncased')
model_fine_tuned = torch.load(fine_tuned_filepath, map_location=torch.device(device_name))

c:\users\krystian kurek\.conda\envs\nlp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly

In [6]:
import tqdm

In [7]:
import os
import glob

In [8]:
models = {'model_pretrained': model_pretrained, 'model_fine_tuned': model_fine_tuned}


In [16]:
concepts = pd.read_csv(concepts_filepath)

In [15]:
for model_name, model in models.items():
    concepts_emb = return_embeddings_for_concepts(concepts['STR'],model)
    concepts_emb['concept_name'] = concepts['STR']
    concepts_emb[['concept_name'] + list(range(len(concepts_emb.columns)-1))]
    filename = embeddings_output_filepath.replace('.csv',f'_{model_name}.csv')
    concepts_emb.to_csv(filename, index=False)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  9.75it/s]


In [23]:
from collections import Counter
Counter(concepts[['CUI', 'SAB']].apply(tuple, axis=1)).most_common()

[(('C4327773', 'GO'), 185),
 (('C4327774', 'GO'), 185),
 (('C3257165', 'NDDF'), 147),
 (('C3823896', 'GO'), 144),
 (('C3823933', 'GO'), 144),
 (('C2316418', 'NDDF'), 138),
 (('C0938061', 'NDDF'), 115),
 (('C1300015', 'NDDF'), 108),
 (('C4327115', 'GO'), 100),
 (('C3160639', 'RXNORM'), 99),
 (('C3162670', 'RXNORM'), 98),
 (('C3195254', 'RXNORM'), 98),
 (('C0023434', 'MSH'), 97),
 (('C4236249', 'GO'), 92),
 (('C0982792', 'NDDF'), 88),
 (('C0023374', 'MSH'), 87),
 (('C3652291', 'NDDF'), 86),
 (('C0689149', 'NDDF'), 84),
 (('C1562536', 'NDDF'), 84),
 (('C0979242', 'NDDF'), 82),
 (('C4048860', 'NDDF'), 81),
 (('C4049028', 'NDDF'), 79),
 (('C0973757', 'NDDF'), 78),
 (('C1269927', 'NDDF'), 76),
 (('C3895373', 'GO'), 76),
 (('C3895399', 'GO'), 76),
 (('C2242628', 'MSH'), 74),
 (('C1576776', 'NDDF'), 73),
 (('C0937479', 'NDDF'), 72),
 (('C0011368', 'MSH'), 72),
 (('C0085172', 'MSH'), 72),
 (('C3823971', 'GO'), 72),
 (('C3823990', 'GO'), 72),
 (('C4236267', 'GO'), 72),
 (('C4236268', 'GO'), 72),

In [26]:
len(Counter(concepts[['CUI', 'SAB']].apply(tuple, axis=1)).most_common())

4397636

In [27]:
len(set(concepts['CUI']))

3765748

In [25]:
concepts[(concepts['CUI'] == 'C4327773') & (concepts['SAB'] == 'GO')]['STR']

6680804    positive regulation of Wnt-mediated midbrain d...
6680805    up regulation of Wnt receptor signaling pathwa...
6680806    up regulation of Wnt receptor signalling pathw...
6680807    up-regulation of Wnt receptor signaling pathwa...
6680808    up-regulation of Wnt receptor signalling pathw...
                                 ...                        
6680984    upregulation of Wg signaling pathway involved ...
6680985    upregulation of Wg signalling pathway involved...
6680986    upregulation of Wingless signaling pathway inv...
6680987    upregulation of Wingless signalling pathway in...
6680988    upregulation of Wnt signaling pathway involved...
Name: STR, Length: 185, dtype: object